In [2]:
from dash import dcc, html, Input, Output, dash_table, State
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import dash


################################################ Create the feature engine graphs ###################################################

# Load the dataset
df = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_output\SNCB_ANOMALY_V3.csv", sep=";")
df_weather = pd.read_csv(
    r"C:\\Users\Florian\COURS_ULB\Data_folder_input\temp_weather",
    dtype={'Columns (0)': str},
    low_memory=False
)

# Create new column with index
df_weather['Row_number'] = df_weather.reset_index().index

# Order columns
df_weather = df_weather[['Row_number', 'station', 'round_time', 'temperature', 'rel_hum', 'precipitation',
                        'pressure']]

# Merge both dataframes
df = df.merge(df_weather, on="Row_number", how='left')

# Convert the 'timestamps_UTC' column to datetime format
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

# Load test data
data_197 = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_output\data_197.csv", sep=',')
data_197['timestamps_UTC'] = pd.to_datetime(data_197['timestamps_UTC'])

# Define data2
data2 = data_197

# Define workshop data
d = [
    ["1", "Traction", "Charleroi", "Atelier de traction de Charleroi", "SNCB", "locomotives & railcars",
     "50.400, 4.458"],
    ["2", "Traction", "Kinkempois", "Atelier de traction de Kinkempois", "SNCB", "locomotives & railcars",
     "50.609, 5.583"],
    ["3", "Traction", "Hasselt", "Atelier de traction d'Hasselt", "SNCB", "locomotives & railcars", "50.937, 5.305"],
    ["4", "Traction", "Arlon", "Atelier de traction d'Arlon", "SNCB", "locomotives & railcars", "49.678, 5.817"],
    ["5", "Traction", "Anvers", "Atelier de traction d'Anvers-Nord", "SNCB", "locomotives & railcars",
     "51.294, 4.383"],
    ["6", "Traction", "Schaerbeek", "Atelier de Traction de Schaerbeek", "SNCB", "locomotives & railcars",
     "50.878, 4.379"],
    ["7", "Traction", "Ostende", "Atelier de traction d'Ostende", "SNCB", "locomotives & railcars", "51.216, 2.946"],
    ["8", "Central", "Cuesmes", "Atelier central de Cuesmes", "SNCB", "passenger equipment", "50.446, 3.933"],
    ["9", "Central", "Malines", "Atelier central de Malines", "SNCB", "passenger equipment", "51.057, 4.292"],
    ["10", "Central", "Salzinnes", "Atelier central de Salzinnes", "SNCB", "passenger equipment", "50.468, 4.844"],
    ["11", "Polyvalent", "Melle", "Atelier polyvalent de Melle", "SNCB", "passenger equipment & wagons",
     "51.013, 3.779"],
    ["12", "TGV", "Forest", "Atelier TGV de Forest", "SNCB", "TGV", "50.822, 4.318"]
]
# Create a list of columns
columns = ["ID", "Type", "City", "Place_Name", "Company", "Speciality", "Coordinates"]

# Create a DataFrame from the data and columns
w = pd.DataFrame(d, columns=columns)

# Split the 'Coordinates' column into separate latitude and longitude columns
w[['Latitude', 'Longitude']] = w['Coordinates'].str.split(',', expand=True)
w = w.astype({"Latitude": "float", "Longitude": "float"})

# Calc the time columns
data_197['time'] = data_197['timestamps_UTC'].dt.time
data_197['seconds'] = (
        data_197['timestamps_UTC'].dt.hour * 60 + data_197['timestamps_UTC'].dt.minute) * 60 + data_197[
                          'timestamps_UTC'].dt.second

# how many zeroes
data_0 = data_197[(data_197['RS_E_InAirTemp_PC1'] == 0)
                  | (data_197['RS_E_InAirTemp_PC2'] == 0)
                  | (data_197['RS_E_OilPress_PC1'] == 0)
                  | (data_197['RS_E_OilPress_PC2'] == 0)
                  | (data_197['RS_E_WatTemp_PC1'] == 0)
                  | (data_197['RS_E_WatTemp_PC2'] == 0)
                  | (data_197['RS_T_OilTemp_PC1'] == 0)
                  | (data_197['RS_T_OilTemp_PC1'] == 0)
                  | (data_197['RS_E_RPM_PC1'] == 0)
                  | (data_197['RS_E_RPM_PC2'] == 0)
                  ]
(data_0 == 0).astype(int).sum(axis=0)

data3 = w.loc[(w['Longitude'] >= data2['lon'].min())
              & (w['Longitude'] <= data2['lon'].max())
              & (w['Latitude'] <= data2['lat'].max())
              & (w['Latitude'] >= data2['lat'].min())]


#Define near_workshop_variable
near_workshop_197 = data_197.loc[(np.abs(data_197['lat'] - data3['Latitude'][0]) < 0.02) 
           & 
           (np.abs(data_197['lon'] - data3['Longitude'][0]) < 0.02)]
near_workshop_197 = near_workshop_197[near_workshop_197['speed_kph'] < 5]
workshop = near_workshop_197[near_workshop_197['date'] == '2023-01-26']
data_cur = data_197[data_197['date'] == '2023-01-26']

# Load anomaly data
anomaly_df = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_output\SNCB_ANOMALY.csv", sep=";")
global_df = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_input\SNCB_DATA_TRAIN_TRAINING_V2.csv",
                        sep=";", low_memory=False)

####################################################### Create the top 5 ###################################################

# Read the CSV file into a DataFrame
top5_trains_df = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_output\Top5_Trains.csv")

# Drop the first column
top5_trains_df = top5_trains_df[['Train ID', 'Anomaly Count']]

# Convert the DataFrame to a Dash table representation
top5_trains_table = dash_table.DataTable(
    id='top5-trains-table',
    data=top5_trains_df.to_dict('records'),
    columns=[
        {'name': 'Train ID', 'id': 'Train ID'},
        {'name': 'Anomaly Count', 'id': 'Anomaly Count'}
    ],
    style_table={'font-family': 'Calibri', 'text-align': 'center'},
    style_cell={'font-family': 'Calibri'},
    style_header={
        'fontWeight': 'normal',
        'font-family': 'Calibri'
    }
)

# Create pie chart for anomalous trains
pie_chart = px.pie(
    data_frame=anomaly_df,
    names='Mapped_veh_id',
    #title='Number of Anomalous Trains',
    hole=0.4
)

# Update layout for anomalous trains pie chart
pie_chart.update_layout(
    title=dict(text="Anomalies percentage by trains", font=dict(size=15), automargin=True, yref='paper'),
    plot_bgcolor="#202020",
    margin=dict(l=10, r=10, t=10, b=10),
    title_x=0.45,  # Center the title
    title_y=0.11  # Move the title down 5 cm
)

####################################################### Create the bar chart ###################################################

# Create bar chart for the trains
train_ids = anomaly_df['Mapped_veh_id'].unique()

# Create a frequency DataFrame
anomaly_df['Dates'] = pd.to_datetime(anomaly_df['Dates'])
grouped_anomaly_df = anomaly_df.groupby(['Dates', 'Mapped_veh_id'])
count_anomaly_df = grouped_anomaly_df.size().reset_index()
occurrences_df = count_anomaly_df

# Create a list of colors for each Mapped_veh_id
all_veh_ids = occurrences_df['Mapped_veh_id'].unique()
color_palette = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f0e'] * len(all_veh_ids)
color_mapping = dict(zip(all_veh_ids, color_palette))

# Create the bar chart plot
bar_chart = px.bar(
    occurrences_df,
    x="Dates",
    y=occurrences_df.Mapped_veh_id,
    color=occurrences_df['Mapped_veh_id'].apply(lambda x: color_mapping[x]),
)

# Update layout for bar chart
legend_labels = {color_mapping[key]: key for key in color_mapping}
bar_chart.update_layout(
    title="Number of Entries for Each Train",
    margin=dict(l=10, r=10, t=20, b=10),
    title_x=0.5,
    title_y=1.0,
    legend_title="Train ID",
    legend=dict(title=None, orientation="h", y=1.2),
    yaxis_title="Frequency",  # Change Y title to "Frequency"
)

# Replace the default legend labels with train IDs
for i, data in enumerate(bar_chart.data):
    if 'name' in data:
        bar_chart.data[i]['name'] = legend_labels[data['name']]

####################################################### Create the map ###################################################

# Load the dataset
df = pd.read_csv(r"C:\\Users\Florian\COURS_ULB\Data_folder_output\SNCB_ANOMALY_V3.csv", sep=";")
df_weather = pd.read_csv(
    r"C:\\Users\Florian\COURS_ULB\Data_folder_input\temp_weather",
    dtype={'Columns (0)': str},  # Specify the dtype for the first column
    low_memory=False
)

# Create new column with index
df_weather['Row_number'] = df_weather.reset_index().index

# Order columns
df_weather = df_weather[['Row_number', 'station', 'round_time', 'temperature', 'rel_hum', 'precipitation',
                        'pressure']]

# Merge both dataframes
df = df.merge(df_weather, on="Row_number", how='left')

# Convert the 'timestamps_UTC' column to datetime format
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

# Create a dropdown with train IDs
train_dropdown = dcc.Dropdown(
    id='train-dropdown',
    options=[{'label': train_id, 'value': train_id} for train_id in df['Mapped_veh_id'].unique()],
    value=df['Mapped_veh_id'].iloc[0],  # Set default value to the first train ID
    style={'width': '50%'}
)

# Create a dropdown with dates
date_dropdown = dcc.Dropdown(
    id='date-dropdown',
    options=[{'label': date.strftime('%Y-%m-%d'), 'value': date.strftime('%Y-%m-%d')} for date in sorted(df['timestamps_UTC'].dt.date.unique())],
    value=df['timestamps_UTC'].dt.date.iloc[0].strftime('%Y-%m-%d'),  # Set default value to the first date
    style={'width': '50%'}
)

# Create a hidden range slider for selecting the time of day range
time_of_day_slider = dcc.RangeSlider(
    id='hidden-time-of-day-slider',
    min=0,
    max=23,
    step=1,
    marks={i: f"{i}:00" for i in range(0, 24)},
    value=[0, 23]
)

# Set the initial coordinates to Belgium with adjusted zoom and center
initial_coordinates = {"lat": 50.5851, "lon": 4.7883}  # Slightly south of the center of Belgium
initial_zoom = 6.9  # Decrease the zoom level by 20%
        
###################################################### Create the app ###################################################
      
# Create the Dash app
app = dash.Dash(__name__)

###################################################### Create the layout ###################################################
  
# Set the initial coordinates to Belgium with adjusted zoom and center
initial_coordinates = {"lat": 50.5851, "lon": 4.7883}  # Slightly south of the center of Belgium
initial_zoom = 6.9  # Decrease the zoom level by 20%

# Set the layout properties
layout = go.Layout(
    mapbox_style="carto-positron",
    mapbox_zoom=initial_zoom,
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox_center=initial_coordinates
)

# Define the main title and line separator with images
main_title = html.Div(
    [
        html.Img(src="assets/ULB_Logo.png", width="200", height="50"), 
        html.H1("INFO-H423 - Data Mining - 202324", style={"font-weight": "bold"}),
        html.Hr(style={"border-top": "2px solid #A9A9A9"}), 
    ],
    style={"text-align": "center"}
)

# Define the subtitle
subtitle = html.Div(
    [
        html.H3("SNCB Train Anomalies Analysis", style={"font-weight": "bold", "margin-top": "10px"}),
    ],
    style={"text-align": "center"}
)

# Define the app layout
app.layout = html.Div([
    main_title,  # Include the main title at the beginning
    subtitle,   # Include the subtitle
    dcc.Tabs([
        dcc.Tab(label='Main Visualization', children=[
            # Row 1: Pie chart and table
            html.Div([
                # Anomaly pie chart
                html.Div([
                    dcc.Graph(figure=pie_chart,
                              style={'width': '50%', 'display': 'inline-block', 'float': 'right'}),
                    html.H2("Top 5 Trains with the Most Anomalies", style={'font-family': 'Calibri'}),
                ]),

                # Table for top 5 trains
                html.Div([
                    top5_trains_table,
                ], style={'display': 'inline-block', 'width': '50%', 'text-align': 'center'})
            ], style={'margin-bottom': '25%'}),

            # Row 2: Bar chart
            html.Div([
                dcc.Graph(figure=bar_chart,
                          style={'width': '100%', 'display': 'block', 'height': '50%',
                                 'title': {'font-family': 'Calibri'}}),
            ]),
        ]),
        dcc.Tab(label='Train Technical Visualization', children=[
            # Full-width dropdown and Graph
            html.Div([
                # Dropdown and Graph
                html.Div([
                    html.Div([
                        train_dropdown,  # Dropdown for selecting train ID
                        date_dropdown,  # Reusing the same date-dropdown for the map
                        time_of_day_slider,  # Hidden range slider for selecting time of day
                    ], style={'width': '50%', 'display': 'inline-block', 'margin': 'auto'}),
                    html.Div([
                        dcc.Graph(id='graph')
                    ], style={'width': '100%', 'display': 'inline-block'}),
                ]),
            ]),
            # Right side: Map
            html.Div([
                dcc.Graph(id='map-graph', figure={'layout': layout})  # Display the map
            ])
        ]),
    ]),
    # Bottom line
            html.Hr(style={"border-top": "2px solid #A9A9A9", "margin-top": "10px", "margin-bottom": "10px"}),
            html.P("The contributors to this project are : Assonfack Deumani Patrice, Kashaeva Gulnur, Leclercq Florian, Worrall Stephen  .",
                   style={"text-align": "center", "font-style": "italic"}),
])

# Define the callback to update the graph based on the selected date
@app.callback(
    Output('graph', 'figure'),
    [Input('date-dropdown', 'value')]
)
def update_graph(selected_date):
    workshop = near_workshop_197[near_workshop_197['date'] == selected_date]
    data_cur = data_197[data_197['date'] == selected_date]

    # Create subplots with titles
    fig = sp.make_subplots(rows=6, cols=2, shared_xaxes=False, vertical_spacing=0.1, subplot_titles=[
        'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2',
        'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2',
        'RS_E_RPM_PC1', 'RS_E_RPM_PC2',
        'speed_kph', 'speed_kph',
        'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2',
        'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2'
    ], x_title='Time')

    data = [
        ['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2'],
        ['RS_E_OilPress_PC1', 'RS_E_OilPress_PC2'],
        ['RS_E_RPM_PC1', 'RS_E_RPM_PC2'],
        ['speed_kph', 'speed_kph'],
        ['RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2'],
        ['RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']
    ]

    # Add traces for the selected date
    for i in range(6):
        for j in range(2):
            fig.add_trace(
                go.Scatter(x=workshop['timestamps_UTC'], y=workshop[data[i][j]], mode='markers',
                           marker=dict(color='red'),
                           name=f'Workshop {data[i][j]}'),
                row=i + 1, col=j + 1
            )
            fig.add_trace(
                go.Scatter(x=data_cur['timestamps_UTC'], y=data_cur[data[i][j]], mode='markers',
                           marker=dict(color='blue'),
                           name=f'Data {data[i][j]}'),
                row=i + 1, col=j + 1
            )

            # Apply time formatting to each subplot
            fig.update_xaxes(
                tickformat='%H:%M:%S',  # Format for the tick labels as hours:minutes:seconds
                row=i + 1, col=j + 1
            )

    # Update layout
    fig.update_layout(
        height=1500,
        width=1000,
        title_text=f"Workshop Data for {selected_date}",
        showlegend=True,
    )

    return fig

# Define the callback to update the map based on the selected train ID, date, and hidden time of day slider
@app.callback(
    Output('map-graph', 'figure'),
    [Input('train-dropdown', 'value'),
     Input('date-dropdown', 'value'),
     Input('hidden-time-of-day-slider', 'value')],
    [State('graph', 'relayoutData')]  # Capture relayoutData from the graph
)
def update_map(selected_train, selected_date, selected_time_of_day_range, relayout_data):
    # Filter the dataset based on the selected train ID, date, and time of day range
    selected_data = df[
        (df['Mapped_veh_id'] == selected_train) &
        (df['timestamps_UTC'].dt.date == pd.to_datetime(selected_date).date()) &
        (df['timestamps_UTC'].dt.hour >= selected_time_of_day_range[0]) &
        (df['timestamps_UTC'].dt.hour <= selected_time_of_day_range[1])
    ]

    # Create a scattermapbox trace with temperature-based color scale
    scattermapbox_trace = go.Scattermapbox(
        lon=selected_data['lon'],
        lat=selected_data['lat'],
        mode="markers",
        marker=dict(
            size=10,
            color=selected_data['temperature'],  # Use temperature for color scale
            colorscale='Viridis',  # Choose a colorscale
            cmin=-5,  # Set the minimum value for the color scale
            cmax=35,  # Set the maximum value for the color scale
            colorbar=dict(title='Temperature (°C)')  # Add a colorbar title
        ),
    )

    # Create a new figure and update its layout with the scattermapbox trace
    map_fig = go.Figure(data=[scattermapbox_trace], layout=layout)

    # Apply relayoutData from the graph if available
    if relayout_data:
        map_fig.update_layout(relayout_data)

    return map_fig

# Run the app
if __name__ == "__main__":
    app.run_server(port=8054, debug=False)